<center><h1  style="color:white; background-color:#000000; border-radius: 0px; padding:25px;"> Rules extraction </h1></center>

This notebook illustrates rules extraction on iris data set from various tree-based models.

In [ ]:
import os
os.chdir('../')
import time

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, accuracy_score,roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier

from extract_rules.extractors import DtreeExtractorClassifier,SirusClassifier,GbExtractorClassifier
from extract_rules.visualization import show_rules


## Load data :

In [ ]:
iris = load_iris()
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

## Single tree : 

In [ ]:
# Usual DecisionTree
clf_dtree = DecisionTreeClassifier(max_depth=10, random_state=0)
clf_dtree.fit(X_train, y_train)
y_pred_dtree = clf_dtree.predict(X_test)
y_pred_proba_dtree = clf_dtree.predict_proba(X_test)
print('PR AUC :', average_precision_score(y_test, y_pred_proba_dtree))
print('ROC AUC :', roc_auc_score(y_test, y_pred_proba_dtree,average='micro',multi_class='ovr'))
print('Accuracy :', accuracy_score(y_test, y_pred_dtree))

In [ ]:
## DecisionTree rules extraction
STree = DtreeExtractorClassifier(max_depth=2,p0=0.1,quantile=10, random_state=0)
STree.fit(X_train,y_train)
y_pred_sirus = STree.predict(X_test)
y_pred_proba_sirus = STree.predict_proba(X_test)
print('PR AUC :', average_precision_score(y_test, y_pred_proba_sirus))
print('ROC AUC :', roc_auc_score(y_test, y_pred_proba_sirus,average='micro',multi_class='ovr'))
print('Accuracy :', accuracy_score(y_test, y_pred_sirus))

In [ ]:
STree.feature_names_in_ = ['sepal length','sepal width','petal length','petal width']
show_rules(STree,max_rules=16)

## Random forest : 

In [ ]:
# Usual RandomForestClassifier
clf_rf = RandomForestClassifier(max_depth=10, random_state=0)
start = time.time()
clf_rf.fit(X_train, y_train)
end = time.time()
y_pred_dtree = clf_rf.predict(X_test)
y_pred_proba_dtree = clf_rf.predict_proba(X_test)
print('PR AUC :', average_precision_score(y_test, y_pred_proba_dtree))
print('ROC AUC :', roc_auc_score(y_test, y_pred_proba_dtree,average='micro',multi_class='ovr'))
print('Accuracy :', accuracy_score(y_test, y_pred_dtree))
time_2 = end-start
print('Fitting time = ',time_2 ,'s')

In [ ]:
## RandomForestClassifier rules extraction
RFSirus = SirusClassifier(n_estimators=1000,max_depth=4,quantile=10,p0=0.01, random_state=0,splitter="quantile")
start = time.time()
RFSirus.fit(X_train,y_train)
end = time.time()
y_pred_sirus = RFSirus.predict(X_test,to_add_probas_outside_rules=True)
y_pred_proba_sirus = RFSirus.predict_proba(X_test,to_add_probas_outside_rules=True)
print('PR AUC :', average_precision_score(y_test, y_pred_proba_sirus))
print('ROC AUC :', roc_auc_score(y_test, y_pred_proba_sirus,average='micro',multi_class='ovr'))
print('Accuracy :', accuracy_score(y_test, y_pred_sirus))
time_2 = end-start
print('Fitting time = ',time_2 ,'s')

In [ ]:
RFSirus.feature_names_in_ = ['sepal length','sepal width','petal length','petal width']
show_rules(RFSirus,max_rules=20,target_class_index=2)

## GB :

In [ ]:
y_train[y_train==2] = 1
y_test[y_test==2] = 1

In [ ]:
y_train

In [ ]:
y_test

In [ ]:
# Usual GradientBoostingClassifier
GB = GradientBoostingClassifier(n_estimators=100,max_depth=3,criterion="squared_error",random_state=19)
start = time.time()
GB.fit(X_train, y_train)
end = time.time()
y_pred_dtree = GB.predict(X_test)
y_pred_proba_dtree = GB.predict_proba(X_test)
print('PR AUC :', average_precision_score(y_test, y_pred_proba_dtree[:,1]))
print('ROC AUC :', roc_auc_score(y_test, y_pred_proba_dtree[:,1]))
print('Accuracy :', accuracy_score(y_test, y_pred_dtree))
time_2 = end-start
print('Fitting time = ',time_2 ,'s')

In [ ]:
# GradientBoostingClassifier rules extraction
GBSirus = GbExtractorClassifier(n_estimators=1000,max_depth=3,p0=0.01,quantile=10,
                            learning_rate=0.1,subsample=1.0,criterion="squared_error",loss="log_loss", 
                            random_state=19,splitter="quantile")
start = time.time()
GBSirus.fit(X_train, y_train)
end = time.time()
y_pred_dtree = GBSirus.predict(X_test,to_add_probas_outside_rules=True)
y_pred_proba_dtree = GBSirus.predict_proba(X_test,to_add_probas_outside_rules=True)
print('PR AUC :', average_precision_score(y_test, y_pred_proba_dtree[:,1]))
print('ROC AUC :', roc_auc_score(y_test, y_pred_proba_dtree[:,1]))
print('Accuracy :', accuracy_score(y_test, y_pred_dtree))
time_2 = end-start
print('Fitting time = ',time_2 ,'s')

In [ ]:
GBSirus.feature_names_in_ = ['sepal length','sepal width','petal length','petal width']
show_rules(GBSirus,max_rules=25,target_class_index=1)